# Multiple Linear Regression

In [2]:
import csv
import pandas as pd
import numpy as np
import sklearn
from sklearn import *

/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will 

In [20]:
pd.set_option('display.max_columns', 65)

df = pd.read_csv('clean_data.csv', low_memory=False)

Variables in the Cleaned Data:

Location & Zoning Data (Potential Categorical X):
<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <ul>
         <li>fips</li>
          <li>latitude</li>
        <li>longitude</li>
         <li>propertycountylandusecode</li>
         
         
         </ul>
    </div>
    <div style="display: inline-block;">
        <ul>
        <li>propertylandusetypeid</li>
         <li>rawcensustractandblock</li>
         <li>regionidcity</li>
         <li>zipcode</li>
        </ul>
    </div>
</div>

Categorical X:

<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <ul>
         <li>airconditioningtypeid</li>
         <li>hashottuborspa</li>
         <li>heatingorsystemtypeid</li>
        <li>fireplaceflag</li>
        <li>assessmentyear</li>
        </ul>
    </div>
    <div style="display: inline-block;">
        <ul>
        <li>taxdelinquencyflag</li>
        <li>setyear</li>
        <li>pooltype</li> 
        <li>transactiondate</li>
        </ul>
    </div>
</div>


Numerical X:
<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <ul>
        <li>bedroomcnt</li>
        <li>buildingqualitytypeid</li>
        <li>calculatedbathnbr</li>
        <li>calculatedfinishedsquarefeet</li>
        <li>fireplacecnt</li>
        <li>fullbathcnt</li>
        <li>garagecarcnt</li>
        <li>garagetotalsqft</li>
        <li>lotsizesquarefeet</li>
        <li>poolcnt</li>
        </ul>
    </div>
    <div style="display: inline-block;">
        <ul>
        <li>roomcnt</li>
        <li>unitcnt</li>
        <li>yearbuilt</li>
        <li>numberofstories</li>
        <li>structuretaxvaluedollarcnt</li>
        <li>taxvaluedollarcnt</li>
        <li>landtaxvaluedollarcnt</li>
        <li>taxamount</li>
        <li>taxdelinquencyyear TO REMOVE</li> 
        </ul>
    </div>
</div>
           
                        
Y:
* logerror

In [21]:
def hottub(a):
    if a:
        return 1
    else:
        return 0
    
df['hashottuborspa'] = df['hashottuborspa'].map(lambda a: hottub(a))

def fire(a):
    if a:
        return 1
    else:
        return 0

df['fireplaceflag'] = df['fireplaceflag'].map(lambda a: fire(a))

def tax(a):
    if a=='Y':
        return 1
    else:
        return 0

df['taxdelinquencyflag'] = df['taxdelinquencyflag'].map(lambda a: tax(a))

fips = pd.get_dummies(df['fips'])
df = df.join(fips)

yr = pd.get_dummies(df['setyear'])
df = df.join(yr)

ac = pd.get_dummies(df['airconditioningtypeid'], prefix='ac')
df = df.join(ac)

heat = pd.get_dummies(df['heatingorsystemtypeid'], prefix='heat')
df=df.join(heat)

df.head()


In [30]:
msk = np.random.rand(len(df)) < 0.8

In [31]:
train = df[msk]
test = df[~msk]

In [29]:
train.describe()

,parcelid,airconditioningtypeid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,propertylandusetypeid,rawcensustractandblock,regionidcity,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,logerror,setyear,pooltype,zipcode
count,1.341560e+05,134156.000000,134156.000000,134156.000000,134156.000000,134156.000000,134156.000000,134156.000000,134156.000000,134156.000000,134156.000000,134156.000000,134156.000000,134156.000000,1.341560e+05,134156.000000,134156.000000,1.341560e+05,131491.000000,132793.000000,134156.000000,134156.000000,134156.000000,1.341560e+05,1.341560e+05,134156.000000,1.341560e+05,134156.000000,3772.00000,134156.000000,134156.000000,134156.000000,134156.000000
mean,1.299581e+07,3.973836,3.042458,6.014671,2.309200,1778.066445,6048.836802,0.126979,2.242867,0.598810,157.131235,3.209294,34.006774,-118.201542,2.600041e+04,0.203159,261.829862,6.049147e+13,33693.751306,5.616663,1.068510,1968.603342,1.099354,1.842371e+05,4.730560e+05,2015.462327,2.892073e+05,5991.370953,2013.78579,0.014165,2016.462327,1.450252,91594.184740
std,3.035481e+06,2.243631,1.151121,1.505300,0.974176,940.662281,20.697520,0.400159,0.961000,0.920488,238.299124,3.031200,0.264953,0.359983,8.536214e+04,0.402351,5.109508,2.053968e+11,46803.520665,1.949434,0.367833,23.697300,0.317841,2.188872e+05,6.089824e+05,0.498581,4.521597e+05,7248.094186,1.68155,0.166088,0.498581,2.863796,1082.850401
min,1.071174e+07,1.000000,0.000000,1.000000,1.000000,120.000000,6037.000000,0.000000,1.000000,0.000000,0.000000,1.000000,33.339295,-119.475416,1.670000e+02,0.000000,31.000000,6.037101e+13,3491.000000,1.000000,1.000000,1824.000000,1.000000,4.400000e+01,2.200000e+01,2015.000000,2.200000e+01,19.920000,1999.00000,-4.655420,2016.000000,0.000000,90001.000000
25%,1.154927e+07,1.000000,2.000000,6.000000,2.000000,1184.000000,6037.000000,0.000000,2.000000,0.000000,0.000000,2.000000,33.813120,-118.413371,5.962000e+03,0.000000,261.000000,6.037312e+13,12447.000000,4.000000,1.000000,1953.000000,1.000000,8.273900e+04,2.030000e+05,2015.000000,8.416525e+04,2799.800000,2013.00000,-0.025207,2016.000000,0.000000,90660.000000
50%,1.253920e+07,5.000000,3.000000,6.015461,2.000000,1541.000000,6037.000000,0.000000,2.000000,0.000000,0.000000,2.000000,34.021500,-118.177000,7.200000e+03,0.000000,261.000000,6.037604e+13,25218.000000,5.000000,1.000000,1970.000000,1.000000,1.340510e+05,3.500000e+05,2015.000000,1.978040e+05,4501.380000,2014.00000,0.006000,2016.000000,0.000000,91367.000000
75%,1.421697e+07,5.000000,4.000000,7.000000,3.000000,2100.000000,6059.000000,0.000000,3.000000,2.000000,434.000000,2.000000,34.173325,-117.925847,1.045000e+04,0.000000,266.000000,6.059042e+13,45457.000000,7.000000,1.000000,1987.000000,1.000000,2.140112e+05,5.532890e+05,2016.000000,3.550000e+05,6906.405000,2015.00000,0.039200,2017.000000,0.000000,92673.000000
max,1.676893e+08,13.000000,16.000000,12.000000,18.000000,35640.000000,6111.000000,5.000000,18.000000,24.000000,7339.000000,24.000000,34.818767,-117.554636,2.539984e+06,1.000000,275.000000,6.111009e+13,396556.000000,32.000000,9.000000,2016.000000,4.000000,1.142179e+07,4.906124e+07,2016.000000,4.895220e+07,586639.300000,2015.00000,4.520000,2017.000000,10.000000,93591.000000


In [33]:
lm=linear_model.LinearRegression()
array_train=train.values
array_train[:,33]

array([0.0953, 0.0198, -0.0566, ..., 0.725458273984, 0.043652112878300016,
       -0.0141783454295], dtype=object)

In [99]:
np.any(np.isnan(df.iloc[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30]]))
#df.iloc[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30]].isnull().any()

False

In [100]:
np.all(np.isfinite(df.iloc[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30]]))

True

In [101]:
lm.fit(array_train[:,[5,27,29,30]],array_train[:,33])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [34]:
lm.fit(array_train[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30,11,25,31,38,39,40,41,42,43,44,45,46,47,48,29,49,50,51,52,53,54,55,56,57,58,59,60]], array_train[:,33])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [35]:
lm.intercept_

0.81811383023513928

In [36]:
lm.coef_

array([  1.26444490e-03,  -1.44032346e-03,  -2.35083659e-03,
         1.37234301e-05,  -2.04919799e-03,   2.30619677e-03,
         3.43384451e-03,  -2.47170658e-05,  -1.65018362e-03,
         6.23591153e-03,   3.43618068e-08,  -9.01521842e-03,
        -6.24786857e-04,  -8.63049266e-03,  -1.09950049e-04,
        -2.73989093e-07,   2.99797628e-07,  -2.70592981e-07,
        -3.08535389e-06,  -1.08708889e-02,  -3.80617390e-03,
         2.00684590e-02,  -6.86727521e-03,  -1.53121017e-03,
         8.39848538e-03,  -2.65115288e-03,   2.65115288e-03,
        -5.74274634e-03,   9.21678724e-02,  -4.34769071e-03,
        -9.26392201e-02,   1.60643648e-02,  -5.50258005e-03])

In [37]:
array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30,11,25,31,38,39,40,41,42,43,44,45,46,47,48,29,49,50,51,52,53,54,55,56,57,58,59,60]])

In [39]:
from sklearn.metrics import mean_squared_error
from math import sqrt

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))
 

0.1654261128024481


In [129]:
reg=linear_model.LinearRegression()
reg.fit(array_train[:,[3,4,7,8,14,27]], array_train[:,33])
p = reg.predict(array_test[:,[3,4,7,8,14,27]])
print(sqrt(mean_squared_error(array_test[:,33],p)))

0.16323141652774348


In [ ]:

from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(train.iloc[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30,11,25,31]], train.iloc[:,33])


In [ ]:
selector.support_ 
selector.ranking_


In [8]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(train.iloc[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30,11,25,31]], train.iloc[:,33])


/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [9]:
clf.intercept_

0.00033743435908003411

In [10]:
clf.coef_ #5, 15 26,27,29,30

array([  0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
         1.06361412e-05,  -0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,   1.96030329e-08,  -0.00000000e+00,
        -0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
        -4.00751045e-08,   6.97427202e-08,  -3.82992658e-08,
        -3.37615125e-06,  -0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00])

In [13]:
from sklearn.metrics import mean_squared_error
from math import sqrt


array_test = test.values
array_train = train.values

reg=linear_model.LinearRegression()
reg.fit(array_train[:,[5,15,26,27,29,30]], array_train[:,33])
p = reg.predict(array_test[:,[5,15,26,27,29,30]])
print(sqrt(mean_squared_error(array_test[:,33],p)))

0.1617368586637341


In [ ]:
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)